# Greedy

- problem: jumlah minimal tim dalam constraint yg diberikan
- constraint: is activity compatible with some team and deadline?

In [4]:
"""
https://docs.google.com/spreadsheets/d/1KxWTopBOX0lhMYJ0ij3u7sYs1s871_Vk/edit
"""

class Greedy():
    a = 1
    data = None
    df = None
    
    def __init__(self):
        pass

    def load(self, fname):
        self.df = pd.read_csv(fname)
        return self
    
    def loadSet(self, fname):
        self.load(fname)
        self.data = self.df
        return self

    # df compatible data
    def setData(self, data):
        self.data = data
        return self

    def isFeasible(self):
        return False
    
    def doGreedy(self):
        res = {}
        for i,d in enumerate(self.data):
            print(i)
        return res

g = Greedy()
result = g.loadSet("const-greedy.csv").doGreedy()
print(g.data)
print(result)

In [2]:
# https://www.toptal.com/developers/hastebin/ivecicixut.yaml
"""
function optimasi(){
    
    dur #jumlah hari yang diperlukan untuk pengerjaan
    cost #biaya yang diperlukan
    costcrash #biayapengahancuran 
    expdaycrash #lamanya waktu crash
    dailyreward #reward
    timeleft = 4*5*6  = 120 #waktu kerja selama 6 bulan
    task = #jumlah task

    #time
    #reward = 0
    #task = 0
    if(dur > 0 and timeleft > 0){
        timeleft = timeleft - dur;
        if(timeleft > 0 and expdaycrash > 0){
            timeleft = timeleft - expdaycrash;
            if(timeleft > 0){
                task = task + 1; #initiate task
            }
        }
    }

    #costreward
    if(cost != 0){
        totalcost = cost;
        if(costcrash != 0){
             totalcost = totalcost + costcrash
        }
        reward = dailyreward - totalcost;
    }

    print(task,reward,)
}
"""
# a
"fun"

'fun'